In [3]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import math

# set plot parameters
plt.rcParams.update({'font.size': 16})
plt.rcParams['text.usetex'] = True

In [4]:
# define a function to apply the kalman filter
def filter(A, Q, C, R, x, P, z):
    # predict step of KF (a priori estimate)
    x_pred = A @ x # predicted state
    P_pred = A @ P @ A.T + Q # predicted state covariance matrix
    
    # update step of KF (a posteriori estimate)
    K = P_pred @ C.T @ np.linalg.pinv(C @ P_pred @ C.T + R) # Standard Kalman gain
    x_filt = x_pred + K @ (z - C @ x_pred) # filtered state
    P_filt = P_pred - K @ C @ P_pred# filtered state covariance matrix
    
    return x_filt, P_filt, x_pred, P_pred

def smooth(A, x_filt, P_filt, x_pred, P_pred):
    x_smooth = x_filt.copy()
    P_smooth = P_filt.copy()
    for i in range(len(x_filt)-2, -1, -1):
        J = P_filt[i] @ A.T @ np.linalg.pinv(P_pred[i+1])
        x_smooth[i] = x_filt[i] + J @ (x_smooth[i+1] - x_pred[i+1])
        P_smooth[i] = P_filt[i] + J @ (P_smooth[i+1] - P_pred[i+1]) @ J.T

    return x_smooth[0], P_smooth[0]


In [5]:
# define parameters
N = 10 # lag window size
A = np.array([[1, 1], [0, 1]]) # train matrix
Q = np.array([[0.25, 0.5], [0.5, 1]]) * 0.1**2 # transition covariance matrix
C = np.array([[1, 0]]) # observation matrix
R = np.array([[3]]) # observation covariance matrix

x = np.array([1, -1]) # initial state
P = np.array([[2**2, 0],
                          [0, 1.5**2]]) # initial state covariance matrix

In [6]:
# generate data
np.random.seed(0)
n = 50
t = np.linspace(0, n, n)
z = t + np.random.normal(0, R[0], n)

In [ ]:
x_final = [] # used to store filtered and smoothed states
P_final = [] # used to store filtered and smoothed state covariance matrices

buffer_x = [x]
buffer_P = [P]

buffer_x_pred = []
buffer_P_pred = []

i = 0
while i <= len(z):
    x, P, x_pred, P_pred = filter(A, Q, C, R, x, P, z[i])
    buffer_x.append(x)
    buffer_P.append(P)
    buffer_x_pred.append(x_pred)
    buffer_P_pred.append(P_pred)
    i += 1
    
    if i >= N:
        x, P = smooth(A, buffer_x, buffer_P, buffer_x_pred, buffer_P_pred)
        x_final.append(x)
        P_final.append(P)
        
        # remove the oldest state
        buffer_x.pop(0)
        buffer_P.pop(0)
        buffer_x_pred.pop(0)
        buffer_P_pred.pop(0)